<a href="https://colab.research.google.com/github/ck1972/Geospatial-Deep-Learning/blob/main/Mod2_Lab2c_Modeling_AGBD_1D_CNN_for_AGBD_Prediction_Model1_Mafungautsi_Ver1_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2c. Modeling Aboveground Biomass Density (AGBD) using 1D Convolutional Neural Network (1D CNN) for AGBD Prediction Model**
## Setup
### Install libraries

In [ ]:
# Install some packages
!pip install rasterio
!pip install earthpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.4 MB/s eta 0:00:00


### Import libraries

In [ ]:
# Import libraries
import rasterio
import earthpy.plot as ep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load and prepare data
### Define variables and data paths

In [ ]:
# Define predictor and target variables, and the data paths
FEATURES = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'NDVI', 'SAVI', 'RESI']
LABEL = ['agbd']
SAMPLE_PATH = '/content/drive/My Drive/Maf_Datasets/TA_AGDD_GEDI_L4A_2022.csv' # With filtered agbd
IMAGE_PATH = '/content/drive/My Drive/Maf_Datasets/S2_predictors_2022.tif'

### Load and visualize Sentinel-2 image

In [ ]:
# Load image
image = rasterio.open(IMAGE_PATH)
bandNum = image.count
height = image.height
width = image.width
crs = image.crs
transform = image.transform
shape = (height, width)

image_vis = []
for x in [5, 6, 4]:
  image_vis.append(image.read(x))
image_vis = np.stack(image_vis)

plot_size = (8, 8)
ep.plot_rgb(
  image_vis,
  figsize=plot_size,
  stretch=True,
)

##  Load sample data

In [ ]:
# Read sample
samples = pd.read_csv(SAMPLE_PATH)[FEATURES + LABEL]
samples

## Split and prepare data

In [ ]:
# Scale input features
scaler = MinMaxScaler()
samples[FEATURES] = scaler.fit_transform(samples[FEATURES])

# Extract features and label
X = samples[FEATURES].values
y = samples[LABEL].values

# Reshape features to (samples, features, 1) for Conv1D
X = X.reshape((X.shape[0], X.shape[1], 1))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Build and train sequential deep learning model
### Model architecture

In [ ]:
# Define 1D CNN Model
input_layer = Input(shape=(X.shape[1], 1))

x = Conv1D(filters=64, kernel_size=2, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Conv1D(filters=32, kernel_size=2, activation='relu')(x)
x = Dropout(0.3)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(1)(x)

model = Model(inputs=input_layer, outputs=output)

### Display the Keras model

In [ ]:
# Import the utility function to visualize Keras models
from tensorflow.keras.utils import plot_model
# Generate a visual representation of the Keras model architecture
plot_model(model, show_shapes=True, show_layer_names=True, dpi=70)

## Compile the model

In [ ]:
# Compile
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
)

## Train the model

In [ ]:
# Train the Model
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=300,
    callbacks=[early_stop],
    verbose=1
)

## Evaluate model performance
### Training curves

In [ ]:
# Plot Training History
history_df = pd.DataFrame(history.history)
plt.figure(figsize=(10, 6))
plt.plot(history_df['rmse'], label='Train RMSE')
plt.plot(history_df['val_rmse'], label='Validation RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.title('Training and Validation RMSE')
plt.legend()
plt.grid(True)
plt.show()

### Test data evaluation

In [ ]:
# Evaluate the Model
y_pred = model.predict(X_test).flatten()

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"\n 1D CNN Model Evaluation")
print(f"R² Score: {r2:.3f}")
print(f"MSE: {mse:.3f}")
print(f"RMSE: {rmse:.3f}")

327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

 1D CNN Model Evaluation
R² Score: 0.508
MSE: 0.032
RMSE: 0.179


### Plot predictions vs. true values

In [ ]:
# Plot Prediction vs. True
plt.figure(figsize=(7, 7))
plt.scatter(y_test, y_pred, alpha=0.5, edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('True AGBD (Mg/ha)')
plt.ylabel('Predicted AGBD (Mg/ha)')
plt.title('1D CNN: True vs Predicted AGBD')
plt.grid(True)
plt.show()

## Predict AGBD map

In [ ]:
# Read all 12 bands from image
image_input = []
for x in range(12):  # assuming 12 features used for training
    image_input.append(image.read(x + 1))  # rasterio is 1-indexed

image_input = np.stack(image_input)  # shape: (12, H, W)
image_input_flat = image_input.reshape(12, -1).T  # shape: (H*W, 12)
image_input_cnn = image_input_flat.reshape(-1, 12, 1)  # shape: (H*W, 12, 1)

# Predict in batches
prediction_flat = model.predict(image_input_cnn, batch_size=4096) * 90  # back to original AGBD scale
prediction_map = prediction_flat.reshape(shape[0], shape[1])

# Visualize
ep.plot_bands(prediction_map, cmap='YlGn', figsize=(10, 10), title='Predicted AGBD Map (1D CNN)')

## Save AGB map

In [ ]:
# Save file to drive
output_path = '/content/drive/MyDrive/Maf_Datasets/AGBD_1DCNN_2022.tif'

with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=shape[0],
    width=shape[1],
    count=1,
    dtype='float32',
    crs=image.crs,
    transform=image.transform
) as dst:
    dst.write(prediction_map.astype('float32'), 1)

print(f"Saved AGBD prediction map to: {output_path}")

✔️ Saved AGBD prediction map to: /content/drive/MyDrive/Maf_Datasets/AGBD_1DCNN_2022.tif
